In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df.shape

(20800, 5)

In [5]:
### dropping the NULL values
df = df.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
### Get the independent Features
x = df.drop('label',axis=1)
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
### Get the Dependent Features
y = df['label']

In [9]:
x.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [12]:
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [13]:
### vocabulary size
voc_size = 5000

### one hot representation

In [14]:
messages = x.copy()

In [15]:
messages.title[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    print(review)
    
    review = [ps.stem(words) for words in review if not words in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    
    

0
['house', 'dem', 'aide', 'we', 'didn', 't', 'even', 'see', 'comey', 's', 'letter', 'until', 'jason', 'chaffetz', 'tweeted', 'it']
1
['flynn', 'hillary', 'clinton', 'big', 'woman', 'on', 'campus', 'breitbart']
2
['why', 'the', 'truth', 'might', 'get', 'you', 'fired']
3
['civilians', 'killed', 'in', 'single', 'us', 'airstrike', 'have', 'been', 'identified']
4
['iranian', 'woman', 'jailed', 'for', 'fictional', 'unpublished', 'story', 'about', 'woman', 'stoned', 'to', 'death', 'for', 'adultery']
5
['jackie', 'mason', 'hollywood', 'would', 'love', 'trump', 'if', 'he', 'bombed', 'north', 'korea', 'over', 'lack', 'of', 'trans', 'bathrooms', 'exclusive', 'video', 'breitbart']
6
['beno', 't', 'hamon', 'wins', 'french', 'socialist', 'party', 's', 'presidential', 'nomination', 'the', 'new', 'york', 'times']
7
['a', 'back', 'channel', 'plan', 'for', 'ukraine', 'and', 'russia', 'courtesy', 'of', 'trump', 'associates', 'the', 'new', 'york', 'times']
8
['obama', 's', 'organizing', 'for', 'action', 

88
['technocracy', 'the', 'real', 'reason', 'why', 'the', 'un', 'wants', 'control', 'over', 'the', 'internet']
89
['american', 'drivers', 'regain', 'appetite', 'for', 'gas', 'guzzlers', 'the', 'new', 'york', 'times']
90
['hillary', 'clinton', 'builds', 'million', 'war', 'chest', 'doubling', 'donald', 'trump', 'the', 'new', 'york', 'times']
91
['trump', 'catches', 'what', 'sick', 'reporter', 'snuck', 'in', 'interview', 'has', 'priceless', 'response']
92
['all', 'senators', 'contacted', 'russian', 'government', 'this', 'week']
93
['images', 'that', 'perfectly', 'illustrate', 'the', 'struggle', 'against', 'the', 'dakota', 'access', 'pipeline']
94
['washington', 'state', 'takes', 'refugees', 'muslim', 'rest', 'of', 'country', 'takes', 'refugees', 'muslim', 'breitbart']
95
['ncaa', 'big', 'keeps', 'watchful', 'eye', 'on', 'texas', 'bathroom', 'bill', 'breitbart']
96
['massive', 'espn', 'financial', 'subscriber', 'losses', 'drag', 'down', 'disney', 's', 'first', 'quarter', 'sales', 'breitbar

167
['mr', 'trump', 's', 'wild', 'ride', 'the', 'new', 'york', 'times']
168
['here', 'is', 'how', 'fbi', 'director', 'comey', 'bamboozled', 'the', 'doj', 'congress', 'and', 'the', 'clintons', 'all', 'at', 'once']
169
['foods', 'that', 'naturally', 'unclog', 'arteries', 'and', 'prevent', 'heart', 'attacks']
170
['death', 'of', 'the', 'two', 'state', 'solution']
171
['comment', 'on', 'parents', 'on', 'a', 'date', 'were', 'asleep', 'in', 'car', 'when', 'cops', 'arrived', 'and', 'killed', 'them', 'both', 'by', 'facespace']
172
['donald', 'trump', 's', 'team', 'shows', 'few', 'signs', 'of', 'post', 'election', 'moderation', 'the', 'new', 'york', 'times']
173
['miami', 'beach', 'tries', 'to', 'tame', 'its', 'most', 'raucous', 'street', 'but', 'the', 'fishbowl', 'drinks', 'can', 'stay', 'the', 'new', 'york', 'times']
174
['doctors', 'mysteriously', 'found', 'dead', 'after', 'summit', 'for', 'breakthrough', 'cure', 'for', 'cancer']
175
['donald', 'trump', 'the', 'unsinkable', 'candidate', 'the

253
['brooks', 'trump', 'siding', 'with', 'a', 'foreign', 'leader', 'against', 'the', 'us', 'president', 'on', 'israel', 'and', 'russia', 'breitbart']
254
['there', 's', 'toxic', 'air', 'in', 'your', 'home', 'and', 'this', 'is', 'you', 'can', 'get', 'rid', 'of', 'it', 'naturally']
255
['a', 'connecticut', 'reader', 'reports', 'record', 'voter', 'registration', 'inspired', 'by', 'trump']
256
['germany', 'reacts', 'to', 'merkel', 'trump', 'visit', 'could', 'have', 'been', 'a', 'lot', 'worse', 'the', 'new', 'york', 'times']
257
['justin', 'rose', 'outduels', 'henrik', 'stenson', 'for', 'golf', 'gold', 'medal', 'the', 'new', 'york', 'times']
258
['iceland', 's', 'water', 'cure', 'the', 'new', 'york', 'times']
259
['m', 't', 'a', 'shortens', 'l', 'train', 'shutdown', 'to', 'months', 'the', 'new', 'york', 'times']
260
['what', 'time', 'will', 'the', 'presidency', 'be', 'decided', 'the', 'new', 'york', 'times']
261
['u', 'n', 'relief', 'official', 'calls', 'crisis', 'in', 'aleppo', 'the', 'ap

341
['assange', 'finally', 'afforded', 'opportunity', 'to', 'give', 'statement', 'over', 'rape', 'accusation']
342
['hillary', 's', 'puppet', 'shows', 'just', 'how', 'much', 'hillary', 'cares', 'about', 'god', 'with', 'omission', 'of', 'words']
343
['the', 'stock', 'market', 'has', 'gone', 'so', 'high', 'it', 's', 'a', 'problem', 'the', 'new', 'york', 'times']
344
['john', 'kerry', 'rejects', 'suggestions', 'of', 'u', 's', 'involvement', 'in', 'turkey', 'coup', 'the', 'new', 'york', 'times']
345
['trump', 'aide', 'stephen', 'miller', 'the', 'u', 's', 'has', 'an', 'absolute', 'sovereign', 'right', 'to', 'determine', 'who', 'can', 'and', 'cannot', 'enter', 'the', 'country', 'breitbart']
346
['year', 'old', 'russian', 'girl', 'speaks', 'languages', 'how', 'did', 'she', 'do', 'this']
347
['this', 'powerful', 'corporate', 'lobby', 'is', 'quietly', 'backing', 'hillary', 'and', 'nobody', 's', 'talking', 'about', 'it']
348
['maintaining', 'a', 'sunny', 'spirit', 'in', 'the', 'face', 'of', 'har

...

17856
['the', 'end', 'game', 'closes', 'in', 'on', 'the', 'clintons', 'as', 'the', 'deep', 'state', 'turns', 'massive', 'collection', 'of', 'criminally', 'damning', 'evidence', 'about', 'to', 'be', 'revealed', 'that', 'will', 'bring', 'down', 'the', 'clinton', 'crime', 'family', 'forever']
17857
['sandy', 'hook', 'promise', 'rakes', 'in', 'million', 'video']
17858
['texas', 'dem', 'arrested', 'for', 'alleged', 'dwi', 'after', 'safe', 'driving', 'seminar']
17859
['places', 'to', 'put', 'your', 'hands', 'together', 'the', 'new', 'york', 'times']
17860
['lower', 'yields', 'and', 'agropoisons', 'what', 'is', 'the', 'point', 'of', 'gm', 'mustard', 'in', 'india']
17861
['travel', 'ban', 'dampens', 'persian', 'new', 'year', 'celebration', 'the', 'new', 'york', 'times']
17862
['up', 'in', 'arms', 'trump', 'nation', 'absolutely', 'rejects', 'mitt', 'romney', 'for', 'secretary', 'of', 'state', 'pick']
17863
['a', 'call', 'for', 'unity']
17864
['comment', 'on', 'photos', 'that', 'capture', 'shock

17943
['daily', 'mail', 'begins', 'start', 'funding', 'hate', 'campaign', 'more', 'soon']
17944
['wikileaks', 'confirming', 'clinton', 'was', 'complicit', 'in', 'the', 'murder', 'of', 'ambassador', 'stevens', 'the', 'css', 'printed', 'this', 'allegation', 'four', 'years', 'ago']
17945
['ohio', 'colleges', 'demand', 'students', 'remain', 'defenseless', 'despite', 'campus', 'carry', 'law', 'breitbart']
17946
['republicans', 'who', 'warned', 'of', 'dithering', 'on', 'ebola', 'now', 'hesitate', 'on', 'zika', 'the', 'new', 'york', 'times']
17947
['the', 'word', 'a', 'headline', 'didn', 't', 'need', 'the', 'new', 'york', 'times']
17948
['breaking', 'riot', 'police', 'set', 'up', 'to', 'raid', 'evict', 'standing', 'rock', 'protesters']
17949
['milo', 'to', 'uc', 'berkeley', 'i', 'll', 'be', 'back', 'breitbart']
17950
['cartel', 'massacres', 'rampant', 'corruption', 'taint', 'mexican', 'border', 'state', 'elections']
17951
['how', 'to', 'have', 'a', 'dinner', 'party', 'friends', 'not', 'requir

18035
['shadow', 'of', 'plea', 'deal', 'for', 'david', 'petraeus', 'loomed', 'over', 'hillary', 'clinton', 'email', 'case', 'the', 'new', 'york', 'times']
18036
['with', 'gold', 'hitting', 'look', 'at', 'who', 'is', 'bullish']
18037
['white']
18038
['free', 'summit', 'from', 'health', 'experts', 'teaches', 'about', 'healthy', 'fats', 'to', 'reverse', 'disease', 'and', 'lose', 'weight']
18039
['how', 'apple', 'empowers', 'and', 'employs', 'the', 'american', 'working', 'class', 'the', 'new', 'york', 'times']
18040
['let', 'women', 'abort', 'babies', 'for', 'being', 'the', 'wrong', 'sex', 'says', 'top', 'medical', 'ethicist']
18041
['why', 'american', 'surrogates', 'are', 'in', 'demand', 'for', 'chinese', 'families']
18042
['watch', 'how', 'fast', 'msnbc', 'cuts', 'black', 'woman', 's', 'interview', 'when', 'she', 'says', 'this', 'about', 'trump']
18043
['david', 'brooks', 'whites', 'are', 'voting', 'their', 'gene', 'pool', 'after', 'being', 'ruined', 'by', 'globalization', 'immigration',

18123
['here', 'are', 'the', 'democrats', 'who', 'are', 'skipping', 'trump', 's', 'inauguration', 'breitbart']
18124
['don', 't', 'despair', 'why', 'this', 'election', 'is', 'our', 'greatest', 'blessing']
18125
['muslim', 'refugee', 'locks', 'wife', 'in', 'bedroom', 'sets', 'house', 'on', 'fire', 'watches', 'her', 'burn', 'media', 'silence']
18126
['porn', 'star', 'bitten', 'by', 'shark', 'during', 'underwater', 'shoot']
18127
['my', 'favorite', 'books', 'kehinde', 'wiley', 'the', 'new', 'york', 'times']
18128
['valerie', 'jarrett', 'obama', 's', 'white', 'house', 'has', 'been', 'scandal', 'free', 'breitbart']
18129
['unpythonic']
18130
['here', 'lies', 'a', 'clue', 'in', 'hebrew', 'points', 'to', 'rome', 's', 'medieval', 'jewish', 'cemetery', 'the', 'new', 'york', 'times']
18131
['social', 'justice', 'attorney', 'andrea', 'burton', 'jailed', 'for', 'refusing', 'to', 'remove', 'black', 'lives', 'matter', 'lapel', 'pin']
18132
['reward', 'for', 'facebook', 'killer', 'as', 'manhunt', 'go

18213
['comment', 'on', 'obamacare', 'architect', 'gruber', 'demands', 'larger', 'mandate', 'penalty', 'by', 'marlene']
18214
['wall', 'street', 'journal', 'democrats', 'trying', 'to', 'squelch', 'susan', 'rice', 'unmasking', 'story', 'breitbart']
18215
['lessons', 'in', 'the', 'delicate', 'art', 'of', 'confronting', 'offensive', 'speech', 'the', 'new', 'york', 'times']
18216
['for', 'our', 'freedom', 'and', 'yours']
18217
['dustin', 'johnson', 'withdraws', 'from', 'the', 'masters', 'with', 'a', 'back', 'injury', 'the', 'new', 'york', 'times']
18218
['dhs', 'officers', 'humiliate', 'judges', 'by', 'enforcing', 'immigration', 'laws', 'declares', 'judge', 'breitbart']
18219
['shock', 'video', 'gunman', 'ambushes', 'police', 'inside', 'arizona', 'walmart']
18220
['cyprus', 'why', 'one', 'of', 'the', 'world', 's', 'most', 'intractable', 'conflicts', 'continues', 'the', 'new', 'york', 'times']
18221
['pics', 'thousands', 'march', 'through', 'london', 'in', 'last', 'anti', 'brexit', 'protest

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [23]:
onehot_rep

[[1378, 813, 2973, 2349, 185, 1964, 1591, 4716, 2478, 388],
 [1680, 3168, 1213, 1305, 2445, 2228, 1452],
 [1813, 2464, 2544, 3512],
 [3933, 2942, 935, 23, 212, 3013],
 [4787, 2445, 3977, 1736, 1540, 2305, 2445, 878, 4470, 4637],
 [599,
  1509,
  3022,
  899,
  3169,
  4156,
  1919,
  3528,
  4076,
  504,
  2394,
  3645,
  3304,
  4175,
  1452],
 [3690, 4645, 4514, 4836, 3049, 3456, 258, 1031, 3192, 1818, 4208],
 [2165, 1368, 3136, 2979, 1954, 3109, 4156, 3439, 3192, 1818, 4208],
 [4485, 4909, 1425, 2090, 1128, 3909, 3847, 3604, 4156, 4396],
 [3164, 20, 3373, 3160, 2261, 4177, 545, 3077],
 [2235, 2264, 3485, 4058, 1907, 3091, 247, 2406, 483, 1853, 3864],
 [23, 4460, 185, 3909, 4156, 1954],
 [1052, 3961, 1650, 936, 1979, 3059, 4053, 1620, 2781],
 [1757, 3123, 4473, 2032, 875, 4750, 3541, 3192, 1818, 4208],
 [1727, 290, 851, 2144, 688, 3192, 1818, 4208],
 [515, 2040, 642, 4006, 2394, 786, 2549, 1275, 1116, 2612],
 [2886, 241, 3168],
 [3049, 679, 3087, 753, 4156, 3445, 835, 1452],
 [2653, 

### Embedding Representation

In [26]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding ='pre',maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 4716 2478  388]
 [   0    0    0 ... 2445 2228 1452]
 [   0    0    0 ... 2464 2544 3512]
 ...
 [   0    0    0 ... 3192 1818 4208]
 [   0    0    0 ... 4929 2245 2785]
 [   0    0    0 ... 4900 3801 4502]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1378,
        813, 2973, 2349,  185, 1964, 1591, 4716, 2478,  388])

In [28]:
### creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer ='adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape

(18285, (18285,))

In [30]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [31]:
x_final


array([[   0,    0,    0, ..., 4716, 2478,  388],
       [   0,    0,    0, ..., 2445, 2228, 1452],
       [   0,    0,    0, ..., 2464, 2544, 3512],
       ...,
       [   0,    0,    0, ..., 3192, 1818, 4208],
       [   0,    0,    0, ..., 4929, 2245, 2785],
       [   0,    0,    0, ..., 4900, 3801, 4502]])

In [32]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [50]:
y_train

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x_final,y_final,test_size = 0.33,random_state=0)

### Model Tranning

In [36]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10,batch_size = 64)

Epoch 1/10
192/192 [==============================] - 17s 65ms/step - loss: 0.3360 - accuracy: 0.8424 - val_loss: 0.1920 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1448 - accuracy: 0.9456 - val_loss: 0.1932 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1058 - accuracy: 0.9621 - val_loss: 0.2040 - val_accuracy: 0.9236
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0810 - accuracy: 0.9721 - val_loss: 0.2441 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0587 - accuracy: 0.9807 - val_loss: 0.3070 - val_accuracy: 0.9054
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0434 - accuracy: 0.9856 - val_loss: 0.2944 - val_accuracy: 0.9175
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0294 - accuracy: 0.9895 - val_loss: 0.3997 - val_accuracy: 0.910

### performance Metrics and Accuracy

In [51]:
y_pred  = model.predict_classes(X_test)

C:\Users\HP\Anaconda3\envs\envtensorflow\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [52]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [53]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [54]:
confusion_matrix(y_test,y_pred)

array([[3131,  258],
       [ 265, 2381]], dtype=int64)

In [55]:
accuracy_score(y_pred,y_test)

0.9133388566694284